# <center> <span style="color:red">Music recognition </span> </center>

The purpose of this notebook is to take in music spectrograms and to recognize musical genres

## Import 

In [1]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

## Import Model Linear and MLP 

In [2]:
import sys  
sys.path.insert(0, '../../python')

from linearModel import linearModel
from mlp import MLPModel

## Create Model 

In [3]:
lm = linearModel()
mlp = MLPModel()

## Algorithme 

In [4]:
def import_images_and_assign_labels(folder, label, X, Y):
    for file in os.listdir(folder):
        image_path = os.path.join(folder, file)
        im = Image.open(image_path)
        im = im.resize((8, 8))
        im = im.convert("RGB")
        im_arr = np.array(im)
        im_arr = np.reshape(im_arr, (8 * 8 * 3))
        X.append(im_arr)
        Y.append(label)

In [5]:
def import_dataset():
    dataset_folder = "G:/Programmes/Python/projetAnnuel/data/"
    train_folder = os.path.join(dataset_folder, "train")
    test_folder = os.path.join(dataset_folder, "test")

    X_train = []
    y_train = []
    
    import_images_and_assign_labels(
        os.path.join(train_folder, "Electro"), [1,0,0], X_train, y_train
    )
    import_images_and_assign_labels(
        os.path.join(train_folder, "Metal"), [0,1,0], X_train, y_train
    )
    
    import_images_and_assign_labels(
        os.path.join(train_folder, "Rap"), [0,0,1], X_train, y_train
    )
    
    X_test = []
    y_test = []
    import_images_and_assign_labels(
        os.path.join(test_folder, "Electro"), [1,0,0], X_test, y_test
    )
    import_images_and_assign_labels(
        os.path.join(test_folder, "Metal"), [0,1,0], X_test, y_test
    )
    
    import_images_and_assign_labels(
        os.path.join(test_folder, "Rap"), [0,0,1], X_test, y_test
    )
    
    return (np.array(X_train) / 255.0, np.array(y_train)), \
           (np.array(X_test) / 255.0, np.array(y_test))

In [6]:
def display_value(array):
    print("Sur le dataset de Train")
    cpt = 0
    number = len(array)/3
    for i in array:
        print(cpt," ",np.argmax(i )," ",i)
        cpt+=1
        if(cpt==number):
            cpt=0

In [7]:
def percentage_music_style(array):
    item1 = 0
    item2 = 0
    item3 = 0
    itemGlobal = 0 
    cpt = 0
    for i in array:
        if cpt<10:
            if np.argmax(i) == 0:
                item2 += 10
        elif cpt<20:
            if np.argmax(i) == 1: 
                item1 += 10
        else:
            if np.argmax(i) == 2:
                item3 +=10
        cpt+=1
    itemGlobal = round((item1+item2+item3)/3,2)
        
    print(f"il y a {item2}% de chanson identifié en électro")
    print(f"il y a {item1}% de chanson identifié en métal")
    print(f"il y a {item3}% de chanson identifié en rap")
    print(f"L'accuracy de notre ia est de {itemGlobal}%")

In [8]:
def run_mlp():
    (X_train, y_train), (X_test, y_test) = import_dataset()
    
    resultat1 = mlp.create_mlp_model([192,4,4,3])
    model1 = resultat1[0]
        
    flattened_dataset_inputs = []
    for p in X_train:
        for i in p:
            flattened_dataset_inputs.append(i)
            
    flattened_dataset_outputs = []
    for p in y_train:
        for i in p:
            flattened_dataset_outputs.append(i)
   
    mlp.train_classification_stochastic_gradient_backpropagation(model1,
                                                                flattened_dataset_inputs,
                                                                flattened_dataset_outputs,
                                                                alpha=1e-3,
                                                                iterations_count=200000)

    predicted_outputs = [mlp.predict_mlp_model_classification(model1, p) for p in X_test]
    

        
    #print(predicted_outputs)
    display_value(predicted_outputs)
    percentage_music_style(predicted_outputs)
    print("predicted_outputs",predicted_outputs)
    mlp.save_mlp_classification(model1)
    
    mlp.free_MLP(resultat1)

In [9]:
run_mlp()

Sur le dataset de Train
0   0   [0.4315303564071655, 0.4276469349861145, 0.1317358911037445]
1   1   [0.4367524981498718, 0.5217657089233398, 0.006316592916846275]
2   1   [0.43107694387435913, 0.5840258598327637, -0.05075989291071892]
3   2   [0.12721529603004456, 0.023202605545520782, 0.7907998561859131]
4   0   [0.39748361706733704, 0.2787376344203949, 0.3686962425708771]
5   2   [0.1181141585111618, 0.021811973303556442, 0.7950934171676636]
6   2   [0.38726383447647095, 0.2508661150932312, 0.4140256345272064]
7   1   [0.4351346492767334, 0.5518330931663513, -0.02477031759917736]
8   1   [0.4329368770122528, 0.5717795491218567, -0.041890472173690796]
9   1   [0.4297352731227875, 0.5915180444717407, -0.05552801489830017]
0   1   [0.421970397233963, 0.6233998537063599, -0.06902828812599182]
1   1   [0.42765989899635315, 0.6015493869781494, -0.06098765507340431]
2   1   [0.4292947053909302, 0.5937609672546387, -0.056822389364242554]
3   1   [0.42115920782089233, 0.6260543465614319, -0.

In [10]:
def load_mlp():
    (X_train, y_train), (X_test, y_test) = import_dataset()
    
    resultat2 = mlp.load_mlp_model("../../Save/mlp_192_4_4_3_classification_2021717234046.json")
    model2 = resultat2[0]
        
    predicted_outputs = [mlp.predict_mlp_model_classification(model2, p) for p in X_test]

    display_value(predicted_outputs)
    percentage_music_style(predicted_outputs)
    
    mlp.free_MLP(resultat2)

In [11]:
load_mlp()

Sur le dataset de Train
0   0   [0.7873943448066711, -0.06914258748292923, 0.15763656795024872]
1   0   [0.9144191145896912, -0.09502891451120377, -0.4631306529045105]
2   1   [0.45191049575805664, 0.6695412993431091, 0.009194983169436455]
3   2   [0.10312346369028091, -0.03641502559185028, 0.8543674945831299]
4   0   [0.7527577877044678, -0.09359872341156006, 0.27348265051841736]
5   2   [-0.013380980119109154, -0.07256414741277695, 0.9035106897354126]
6   2   [-0.07423452287912369, 0.02244691364467144, 0.9142631888389587]
7   0   [0.8213911652565002, -0.0452880784869194, 0.010290654376149178]
8   0   [0.860662579536438, -0.09587367624044418, -0.11248285323381424]
9   0   [0.8371292948722839, -0.04813019186258316, -0.0488709919154644]
0   1   [0.23629648983478546, 0.8104368448257446, 0.00112569285556674]
1   1   [0.3492873013019562, 0.7483999729156494, 0.0030205792281776667]
2   1   [-0.05272965133190155, 0.9048637747764587, -0.011044247075915337]
3   1   [-0.10041249543428421, 0.9153